## Packages

In [76]:
import pandas as pd             # Traitement des tableaux de données 

# Chargement des données

Dans un premier temps, l'objectif est de récupérer toutes les valeurs de 2012 à 2022. Les données de 2010 à 2011 (inclus) comprennent des données obsolètes et peu pertinentes pour la construction du modèle. Nous éliminons donc les données des deux premières années.

Dans un second temps, il y a 4 fichiers par an. Il faudra donc récupérer les données et les "merge" afin de construire le jeu de données d'une seule année contenant toutes les informations relatives aux accidents. Cela se traduit par une relation entre le **"*Num_Acc*"** et pour certains fichiers le **"*num_veh*"**.

Ainsi, nous obtenons un fichier complet contenant toutes les colonnes possibles (par exemple, il y aura une colonne **"*secu*"**, et 3 colonnes **"*secu1*"**, **"*secu2*"** et **"*secu3*"**). Il faudra les traiter plus tard.

In [77]:
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
files = ['usagers_', 'vehicules_', 'caracteristiques_', 'lieux_',]
data_path = 'Data/TRAIN/BAAC-Annee-'

full_data = pd.DataFrame()

for year in years:
    
    path_to_BAAC = data_path + year + '/'
    
    year_file = pd.DataFrame()
    
    for file_name in files:        
        path_to_file = path_to_BAAC + file_name + year + '_.csv'
        
        data = pd.read_csv(path_to_file, encoding="latin1", sep=";", low_memory=False)
        data = data.drop(data.columns[0], axis=1)
        
        if file_name == 'usagers_':
            year_file = pd.concat([year_file, data], axis=1)
        else:
            if file_name == 'vehicules_':
                year_file = pd.merge(year_file, data, on=['Num_Acc', 'num_veh'], how='left')
            else:
                year_file = pd.merge(year_file, data, on=['Num_Acc'], how='left')
    
    full_data = pd.concat([full_data, year_file], axis=0)

In [78]:
full_data.shape

(1281392, 59)

Désormais, nous disposons d'un **"*.csv*"** de grandes amplitudes et nous devons comprendre les données ainsi que harmoniser ces dernières au sein du **"*pré-traitement des données*"** :

# Pré-Traitement des données

### Usagers

Les fichiers usagers contiennent globalement des informations relatives auxdits usagers tels que leurs positions, les actions qu'ils faisaient avant l'accident, les dispositifs de sécurité, des données personnelles, etc.

#### Num_Acc, num_veh & id_vehicule

- **Num_Acc** : identifiant de l'accident. Valeur **vitale** pour joindre les données. Cependant, au-delà de la fusion des données, ce numéro n'apporte rien en matière de prédiction. En effet, il est générique et ne transmet pas d'information complémentaire. On se demande alors s'il ne vaudrait pas mieux le supprimer. Dans ce cas-là, on décide de ne pas stocker cette colonne dans l'une des listes traduisant son abandon, car on ne conservera que les variables stockées. 

- **num_veh** : de la même manière, le numéro du véhicule ne semble pas particulièrement pertinent pour la construction d'un modèle de prédiction. Etant donné qu'il n'apporte pas d'informations sur le type de véhicule, et qu'il est propre à un accident et non à un référentiel global, on préfère écarter cette situation.

- **id_vehicule** : cette variable n'est pas apparue dès le début. En effet, il a fallu attendre l'an 2019 pour la voir apparaître. Cependant, il s'avère que cette dernière correpsond au numéro veh au format numérique et non alpha. On décidera donc de ne pas le retenir si elle est présente.

In [79]:
col_to_drop = ['Num_Acc', 'id_vehicule_x', 'id_vehicule_y', 'num_veh']
og_var_number = full_data.shape[1]
new_var_number = og_var_number - len(col_to_drop)

for col in col_to_drop:
    if col in full_data.columns:
        full_data.drop(labels=col_to_drop, axis=1, inplace=True)
        
print(f"Le nombre de variables post suppression est de {full_data.shape[1]}. Selon les calculs, il devrait être de {new_var_number}")

Le nombre de variables post suppression est de 55. Selon les calculs, il devrait être de 55


Une fonction utile permettrait de convertir une variable afin d'éviter la duplication de code :

In [80]:
def convert_float_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    data_frame[col_name] = data_frame[col_name].astype('category')

In [81]:
def convert_float_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)

#### grav

Etant la colonne à prédire, il est évident que nous souhaitons conserver cette dernière. Nous nous penchons sur la qualité de cette dernière. On observe la présence de la valeur '-1' 279 fois. Proportionnellement à la taille des données, on décide donc de supprimer ces quelques lignes ne pouvant les rendre utilisables.

Dans l'exercice de prédiction, il est demandé de convertir cette colonne en binaire (**'*0*'** pour les cas sans dangers [1, 4] et **'*1*'** pour les cas graves [2, 3]). Cette variable sera donc catégorisée en binaire conformément aux prédictions attendues.

In [82]:
# Supprime les valeurs "-1"
full_data = full_data[full_data['grav'] != -1]
full_data = full_data.dropna(subset=['grav'])

# Binarise les valeurs pour les accidents graves et non graves
full_data['grav'] = full_data['grav'].replace({1: 0, 4: 0, 2: 1, 3: 1})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'grav')

full_data['grav'].dtype

CategoricalDtype(categories=[0, 1], ordered=False, categories_dtype=int32)

#### sexe

Certaines études ont pu démontrer des disparités dans les comportements de conduite entre les différents sexes. Nous conservons cette donnée dans l'éventualité où une corrélation existerait entre le sexe, le statut de conducteur et la gravité. Un soupçon se porte sur l'impulsivité et la vitesse de certains conducteurs masculins.

In [83]:
# Binarisation [0, 1] des valeurs de sexe
full_data = full_data[full_data['sexe'] != -1]
full_data['sexe'] = full_data['sexe'].replace({2: 0})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'sexe')

full_data['sexe'].dtype

CategoricalDtype(categories=[0, 1], ordered=False, categories_dtype=int32)

#### An_nais

On dispose ici de l'année de naissance et cette donnée semble pertinente. En effet, en fonction des générations, des politiques de sensibilisation plus ou moins importantes furent appliquées amenant à de potentielles disparités en matière de comportements routiers.

In [84]:
# Supprime les valeurs manquantes
full_data = full_data.dropna(subset=['an_nais'])

# Converti en variable numérique
convert_float_to_int(full_data, 'an_nais')

full_data['an_nais'].dtype

dtype('int32')

#### trajet

Cette variable nous informe sur le motif du déplacement au moment de l'accident. Cela peut renvoyer au trajet entre le domicile et le travail, une promenade, etc. Il se pourrait qu'une corrélation existe entre des trajets récurrents et une augmentation des accidents. En effet, conformément au code de la route, il est indiqué qu'une baisse de la concentration est observée dans les trajets quotidiens.

Afin de normaliser les différentes valeurs, nous nous penchons sur la documentation. Conformément au document de 2005 à 2020 nous disposons des valeurs suivantes : [**-1**, **0**, **1**, **2**, **3**, **4**, **5**, **9**]. Cependant, le document de 2005 à 2018, les valeurs sont différentes : [**1**, **2**, **3**, **4**, **5**, **9**]. Les valeurs suivantes : [**1**, **2**, **3**, **4**, **5**, **9**] sont les mêmes pour les deux référentiels. Néanmoins, les valeurs [**-1**, **0**, **9**] présentent au sein du document de 2005 à 2020 n'apportent pas davantage d'information. Afin de pouvoir traiter avec pertinences ces informations et de manière cohérente, on remplacera les valeurs [**-1**, **0**, **nan**] par [**9**].

In [85]:
# Harmonise les données [1, 2, 3, 4, 5, 9]
full_data['trajet'] = full_data['trajet'].replace({-1: 9, 0: 9})
full_data = full_data.dropna(subset=['trajet'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'trajet')

full_data['trajet'].dtype

CategoricalDtype(categories=[1, 2, 3, 4, 5, 9], ordered=False, categories_dtype=int32)

#### locp

Permet de localiser la position du piéton ce qui est une information importante. En effet, un piéton qui traverse sur un passage piéton sans signalisation lumineuse aura plus de chance de subir des lésions graves, voire plus, qu’un piéton sur un trottoir par exemple. Cependant, en fonction des années, les valeurs peuvent changer pour ce qui est des assignations.

De 2005 à 2018, les valeurs possibles sont les suivantes : [1, 2, 3, 4, 5, 6, 7, 8]. Ces valeurs correspondent aux mêmes situations pour celles de 2005 à 2020, cependant ‘3’ nouvelles catégories sont apparues : [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]. Globalement, [-1, 0, 9] traduisent une absence d’information précise sur la position dudit piéton. On pourra ainsi définir que nos valeurs possibles seront les suivantes : [1, 2, 3, 4, 5, 6, 7, 8, 9] où [-1, 0, nan] prendra la valeur [9].

In [86]:
# On drop la colonne locp qui est trop vide
full_data.drop(labels=['locp'], axis=1, inplace=True)

#### etatp

Cette variable permet de préciser si le piéton accidenté était seul ou non. Il est envisageable, mais pas nécessairement vrai, que la majorité des groupes soient distraits par leurs camarades amenant à une baisse de la vigilance en matière de protection routière. On songera donc à conserver cette catégorie. Sous réserve que cette dernière se révèle impertinente, elle sera supprimée du modèle.

A l'instar de certaines variables, la documentation diffère selon les années et il est nécessaire d'harmoniser cela. Les valeurs possibles pour la documentation de 2005 à 2018 sont : [**1**, **2**, **3**] pour [**seul**, **accompagné**, **en groupe**]. Celles de 2005 à 2020 sont au format suivant : [**-1**, **1**, **2**, **3**] traduisant [**non renseigné**, **seul**, **accompagné**, **en groupe**]. On décidera donc de conserver le format [**-1**, **1**, **2**, **3**] pour la gestion des valeurs inconnues pour l'instant.

In [87]:
# On drop la colonne etatp qui est trop vide
full_data.drop(labels=['etatp'], axis=1, inplace=True)

#### catu

Correspond à la catégorie d'usager. Autrement dit, s'il s'agit d'un conducteur, d'un passager ou bien d'un piéton (ou encore d'un piéton en roller ou en trottinette conformément à la documentation de 2005 à 2018). Il est important de disposer de ces informations en cas de corrélations entre le statut de la victime et son état. On peut soumettre l'hypothèse que ce statut influe sur les situations dans lesquelles les usagers sont impliqués notamment (difficile d'aller à 130 km/h en tant que piéton).

Cependant, il est nécessaire d'harmoniser ces valeurs, car elles différent en fonction des années. D'origine, les valeurs possibles sont les suivantes : [**1**, **2**, **3**, **4**], mais depuis 2018, la catégorie [**4**] évolue vers le fichier "Véhicules" comme catégorie **"*99 - Autre véhicule*"**. Ainsi, une modification envisageable serait de remplacer les valeurs [**4**] par celle des piétons [**3**] **ET** de modifier la valeur associée au sein de la colonne **"*catv*"** par la valeur **"*99*"** :

In [88]:
# Pour les années antérieures à 2019 : catu = 4 -> catv = 99 & catu = 3
full_data.loc[full_data['catu'] == 4, 'catv'] = 99
full_data['catu'] = full_data['catu'].replace({4: 3})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'catu')

full_data['catu'].dtype

CategoricalDtype(categories=[1, 2, 3], ordered=False, categories_dtype=int32)

#### place

Permet de situer la place occupée dans le véhicule par l'usager au moment de l'accident. Cette donnée est importante, car il existe certainement une relation entre la position des victimes et la gravité de l'accident. En effet, en fonction de la position, les dispositifs de sécurité ne sont pas les mêmes. On soupçonne donc une importante majeure de cette variable.
 
Les informations relatives aux positions sont différentes selon les années. Les valeurs possibles pour le document de 2005 à 2018 sont les suivantes : [**1**, **2**, **3**, **4**, **5**, **6**, **7**, **8**, **9**]. Pour le document de 2005 à 2020 : [**1**, **2**, **3**, **4**, **5**, **6**, **7**, **8**, **9**, **10**]. Le nouvel élément correspond à la position du piéton. Une harmonisation se fera où le piéton devra être catégorisé.

In [89]:
# Si c'est un piéton alors sa place est 10 (gestion des différentes réglementations)
full_data.loc[full_data['catu'] == 3, 'place'] = 10
full_data['place'] = full_data['place'].replace({-1: pd.NA})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'place')

full_data['place'].dtype

CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ordered=False, categories_dtype=int32)

#### secu

Ce modèle a fortement évolué au cours du temps en passant d'un format à deux caractères indiquant dans un premier temps le système de sécurité et dans un second les conditions d'utilisation (oui/non/non déterminé). 

Puis dans un nouveau format à 3 colonnes (secu1/secu2/secu3), chacune des colonnes indiquait l'utilisation d'un équipement de sécurité parmi une liste d'éléments comprenant les valeurs suivantes : [**-1**, **0**, **1**, **2**, **3**, **4**, **5**, **6**, **7**, **8**, **9**]. Globalement, [**-1**, **8**, **9**] apportent la même précision (aucune). Ainsi, il est décidé dans un premier temps de rassembler ces trois possibilités en une seule sous la bannière [**8**]. 

Dans un second temps, la première configuration à **"*secu*"** unique sera scindé en **"*3*"** où seul un équipement pourra être notifié. 

Ces décisions sont prises, car nous ne souhaitons pas perdre d'informations (précision accrue sous le format : secu1/secu2/secu3), en effet cette catégorie de variables est très intéressante. Il semble évident que des dispositifs de sécurité viennent minimiser les dégâts subits.

In [90]:
# Fonction pour extraire le premier chiffre renvoyant à l'équipement utilisé
def extract_first_digit(value):
    if pd.notna(value):
        return int(str(value)[0])
    else:
        return 8

In [91]:
# Récupère les lignes où secu1/secu2/secu3 sont NAN (rare cas après 2019, tous les cas avant 2019)
all_nans = full_data[['secu1', 'secu2', 'secu3']].isna().all(axis=1)
filtered_data = full_data[all_nans]

# Conserve seulement le cas où il y a un second digit = 1, sinon applique la valeur 8 (non déterminable)
filtered_data.loc[(filtered_data['secu'] <= 10) | (filtered_data['secu'] % 10 == 2) | (filtered_data['secu'] % 10 == 3), 'secu'] = 8
filtered_data['secu'] = filtered_data['secu'].apply(extract_first_digit)

# Défini dans secu1 le seul équipement noté (1 seul avant 2019) et 8 pour les deux autres
filtered_data['secu1'] = filtered_data['secu']
filtered_data['secu2'] = 8
filtered_data['secu3'] = 8
# Met à jour les colonnes secu/secu1/secu2/secu3
full_data.loc[all_nans, :] = filtered_data

# On peut drop la colonne secu qui ne vaut plus rien
full_data.drop(labels=['secu'], axis=1, inplace=True)

# Dans les cas survenus après 2019
list_secu = ['secu1', 'secu2', 'secu3']

for secu_i in list_secu:
    full_data[secu_i] = full_data[secu_i].replace({-1: 8, 9: 8})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'secu1')
convert_float_to_cat(full_data, 'secu2')
convert_float_to_cat(full_data, 'secu3')

C:\Users\lucas\AppData\Local\Temp\ipykernel_13872\1749354471.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['secu'] = filtered_data['secu'].apply(extract_first_digit)
C:\Users\lucas\AppData\Local\Temp\ipykernel_13872\1749354471.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['secu1'] = filtered_data['secu']
C:\Users\lucas\AppData\Local\Temp\ipykernel_13872\1749354471.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [92]:
full_data['secu1'].dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8], ordered=False, categories_dtype=int32)

In [93]:
full_data['secu2'].dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8], ordered=False, categories_dtype=int32)

In [94]:
full_data['secu3'].dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8], ordered=False, categories_dtype=int32)

#### actp

Concerne l'action du piéton. Diffère de nouveau selon les années. Le premier format de 2005 à 2018 est le suivant : [**0**, **1**, **2**, **3**, **4**, **5**, **6**, **9**]. Pour la version de 2005 à 2020 on a : [**-1**, **0**, **1**, **2**, **3**, **4**, **5**, **9**, **A**, **B**]. De la même manière, on définit que [**-1**, **0**, **9**, **B**, **NA**] apporte la même qualité d'information dans la seconde réforme. Nous décidons de nous calquer sur le format le plus précis soit le second.

Cette catégorie est relativement intéressante, car elle indique si des situations peuvent amener plus ou moins fréquemment à des accidents plus, ou moins graves.

In [95]:
# On drop la colonne actp qui est trop vide
full_data.drop(labels=['actp'], axis=1, inplace=True)

In [96]:
full_data.to_csv('Data/TRAIN_FULL/BAAC_2012_to_2022_V1_to_V4.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.